# Summary
1. We process [Maryland's Global Cropland layer ](https://glad.umd.edu/dataset/croplands)
2. We then export the data for 2019 and 2015 to Google Cloud Storage.

Access cropland data like so:
- 2019 "users/potapovpeter/Global_cropland_2019"

### Future
-

# Setting up the environment

In [ ]:
# Import and/or install libraries
import subprocess, os, gcsfs, json

try:
    import geemap, ee
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])
    import geemap, ee


In [ ]:
# Connect to Google Drive to access files

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Connect to Google Earth Engine if neccessary

service_account = os.environ.get('GOOGLE_SERVICE_ACCOUNT')
credentials = ee.ServiceAccountCredentials(service_account, os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))
ee.Initialize(credentials)

In [ ]:
block_list = list(range(1, 26))

blocks = []
with open("/content/drive/MyDrive/data/blocks.geojson") as f:
    json_data = json.load(f)
    for block_id in block_list:
        for feature in json_data['features']:
            if feature['properties']['id'] == block_id:
                feature['properties']['style'] = {}
                blocks.append(feature)

ee_blocks = [geemap.geojson_to_ee(block) for block in blocks]

In [ ]:
# Specify year, one of [2003, 2007, 2011, 2015, 2019]
year = 2019

In [ ]:
# Prepare layers

# Import cropland
cropland = ee.ImageCollection("users/potapovpeter/Global_cropland_" + str(year)).median().selfMask().toByte()



In [ ]:
# Loop through blocks and export
for id, block in enumerate(ee_blocks):

  if id != 13:

    image_name = 'crop' + str(year) + '_' + str(id)
    fileNamePrefix = 'COGS/crop/' + str(year) + '/' + image_name


    exportConfig = {
        'image': cropland,
        'description': image_name,
        'bucket': 'nature-watch-bucket',
        'fileNamePrefix': fileNamePrefix,
        'scale': 30,
        'maxPixels': 3147395000,
        'region': block,
        'fileFormat': 'GeoTIFF',
        'formatOptions': {'cloudOptimized': True}
    }

    task = ee.batch.Export.image.toCloudStorage(**exportConfig)
    task.start()

In [ ]:
# Or just export one block
id = 13
block = ee_blocks[id]

image_name = 'crop' + str(year) + '_' + str(id)
fileNamePrefix = 'COGS/crop/' + str(year) + '/' + image_name


exportConfig = {
    'image': cropland,
    'description': image_name,
    'bucket': 'nature-watch-bucket',
    'fileNamePrefix': fileNamePrefix,
    'scale': 30,
    'maxPixels': 3147395000,
    'region': block,
    'fileFormat': 'GeoTIFF',
    'formatOptions': {'cloudOptimized': True}
}

task = ee.batch.Export.image.toCloudStorage(**exportConfig)
task.start()

In [ ]:
ee.data.listOperations()

In [ ]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayer(cropland, {}, 'cropland')

Map.setCenter(11.578, 1.600, 12)
Map